# Project 7 

Using dataset (German Credit Data) train to identify good and bad sutomer:

    1.voting classifier (using five different predictors and compare weighted vs unweighted voting results)
    2.train random forest (random features and training set) selections
    3.train adaboost

In [1]:
import numpy as np
import urllib.request as ur
# url with dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
# download the file, here i decide to use the original dataset, instead of the one it has quantiflied.
raw_data = ur.urlopen(url)
text_file = np.loadtxt(raw_data,dtype=str)

In [2]:
#identify the catergory col and the numeric col
numeric = [1,4,7,10,12,15,17]
quantitative = [0,2,3,5,6,8,9,11,13,14,16,18,19]

In [3]:
#this function is used to identify whether a string can be converted into flaot
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [4]:
#separate the category cols from the numeric cols, and conver the numeric cols type from str to float
X_numeric = []
X_catergory = []
for i in range(len(text_file[0])-1):
    if is_number(text_file[0,i]) == True:
        col_numeric = [np.float(num) for num in text_file[:,i]]
        X_numeric.append(col_numeric)
    else:
        X_catergory.append(text_file[:,i])
y = [np.float(num) for num in text_file[:,20]]

In [5]:
#let me design a function to convert the str into integer. 
##my idea is to check the string, if meeting the new kind of string, assign a integer to it.
##then check what integer, a string matches and then convert the string into that integer 
def conver_category(data):   #data should be 1-D here
    catergory = []
    label = {}
    new_label = 0
    for i in range(len(data)):
        if data[i] not in label.keys():
            new_label = new_label + 1
            label[data[i]] = new_label
        catergory.append(label[data[i]])
    return catergory, label

In [6]:
#convert catergory into number 
X_quantitative = []
for i in range(len(X_catergory)):
    catergory, label = conver_category(X_catergory[i])
    X_quantitative.append(catergory)

In [7]:
#try to convert quantitative col into dummy variable
import pandas as pd
X_quantitative_dummy = [pd.get_dummies(X_quantitative[i],drop_first = True) for i in range(len(X_quantitative))]
X_quantitative_dummy = [X_quantitative_dummy[i].values[:, :] for i in range(len(X_quantitative_dummy))]

i think for the scaling part, i should just scale the numeric col, the quantative part should remain the same. Also, i try two case, one is to treat the category column as real number col, another is to turn the category column into dummy variable.

In [8]:
#note that col 2, 5, 8, 11, 13, 16 and 18 is the numeric columns, let's do the standardlization for it.
from sklearn import preprocessing
normalized_X_numeric = preprocessing.scale(np.asarray(X_numeric).T)

In [9]:
#combine them
X_no_dummy = np.hstack((normalized_X_numeric, np.asarray(X_quantitative).T))

X_dummy = normalized_X_numeric
for i in range(len(X_quantitative_dummy)):
    X_dummy = np.hstack((X_dummy, X_quantitative_dummy[i]))

In [10]:
#separate them to train set and test set
num = int(.6*len(X_dummy))
#break the dataset
X_dummy_train = X_dummy[:num]
X_dummy_test = X_dummy[int(num):]

X_no_dummy_train = X_no_dummy[:num]
X_no_dummy_test = X_no_dummy[int(num):]

y_train = np.asarray(y[:num])
y_test = np.asarray(y[int(num):])

In [18]:
for i in range(len(X_dummy_test)):
    if X_dummy_test.size == 0 :
        print ('mistake')

# Voting Classifier

In [12]:
#for voting classifier, i use logistic, naive bayes, knn, svc and decision tree
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression()
nb_clf = GaussianNB()
knn_clf = KNeighborsClassifier()
svc_clf = SVC(probability=True)
CART_clf = DecisionTreeClassifier()

estimators = []
estimators.append(('logistic', log_clf))
estimators.append(('naive_bayes', nb_clf))
estimators.append(('knn', knn_clf))
estimators.append(('svc', svc_clf))
estimators.append(('CART', CART_clf))

#case of using dummy variable 
#case of using unweighte voting
# create the ensemble model
ensemble_hard = VotingClassifier(estimators, voting = 'hard')
ensemble_hard.fit(X_dummy_train, y_train)
ensemble_hard.fit(X_dummy_train, y_train)
predicted_hard = ensemble_hard.predict(X_dummy_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, predicted_hard))
print(metrics.confusion_matrix(y_test, predicted_hard))

#case of using weight voting
ensemble_soft = VotingClassifier(estimators, voting = 'soft')
ensemble_soft.fit(X_dummy_train, y_train)
predicted_soft = ensemble_soft.predict(X_dummy_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, predicted_soft))
print(metrics.confusion_matrix(y_test, predicted_soft))

             precision    recall  f1-score   support

        1.0       0.75      0.92      0.83       277
        2.0       0.63      0.32      0.42       123

avg / total       0.71      0.73      0.70       400

[[254  23]
 [ 84  39]]
             precision    recall  f1-score   support

        1.0       0.78      0.86      0.82       277
        2.0       0.59      0.45      0.51       123

avg / total       0.72      0.73      0.72       400

[[239  38]
 [ 68  55]]


/Users/wangj3475/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/wangj3475/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
#try not dummy variables
ensemble_hard.fit(X_no_dummy_train, y_train)
predicted_hard = ensemble_hard.predict(X_no_dummy_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, predicted_hard))
print(metrics.confusion_matrix(y_test, predicted_hard))

ensemble_soft.fit(X_no_dummy_train, y_train)
predicted_soft = ensemble_soft.predict(X_no_dummy_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, predicted_soft))
print(metrics.confusion_matrix(y_test, predicted_soft))

             precision    recall  f1-score   support

        1.0       0.77      0.84      0.80       277
        2.0       0.54      0.42      0.47       123

avg / total       0.70      0.71      0.70       400

[[233  44]
 [ 71  52]]
             precision    recall  f1-score   support

        1.0       0.77      0.84      0.80       277
        2.0       0.54      0.43      0.48       123

avg / total       0.70      0.71      0.70       400

[[232  45]
 [ 70  53]]


/Users/wangj3475/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/wangj3475/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


i think the weighted result is a little bit better than the weighted result. And also, i think using the dummy variable could improve the result.

# Random Forest

For Random Forest, i could use the dummy varibale for catergories. Moreover, i could use the package of bagging and randomforest, since there is no random train set options in the random forest, and i would like to compare the result between them.

In [22]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 50, max_features=10,
                                 oob_score = True, n_jobs = -1)
rnd_clf.fit(X_dummy_train,y_train)

y_pred_rf = rnd_clf.predict(X_dummy_test)
print(metrics.classification_report(y_test, y_pred_rf))
print(metrics.confusion_matrix(y_test, y_pred_rf))

             precision    recall  f1-score   support

        1.0       0.75      0.91      0.82       277
        2.0       0.62      0.33      0.43       123

avg / total       0.71      0.73      0.70       400

[[252  25]
 [ 82  41]]


In [24]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(splitter = 'random', max_leaf_nodes = 20), 
                            n_estimators = 500, max_features = 10, max_samples = 400, bootstrap = True, n_jobs = -1)
bag_clf.fit(X_dummy_train,y_train)

y_pred_bag = bag_clf.predict(X_dummy_test)
print(metrics.classification_report(y_test, y_pred_bag))
print(metrics.confusion_matrix(y_test, y_pred_bag))

             precision    recall  f1-score   support

        1.0       0.70      0.99      0.82       277
        2.0       0.67      0.03      0.06       123

avg / total       0.69      0.70      0.59       400

[[275   2]
 [119   4]]


The result is terrible for bagging result, seldom it would predict the '2' sucessfully. (is this a problem of using random train set instead of all of them ?)

# AdaBoost

In [25]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 1), n_estimators = 500, algorithm = 'SAMME.R',
                            learning_rate = .5)
ada_clf.fit(X_dummy_train,y_train)
y_pred_ada = ada_clf.predict(X_dummy_test)
print(metrics.classification_report(y_test, y_pred_ada))
print(metrics.confusion_matrix(y_test, y_pred_ada))

             precision    recall  f1-score   support

        1.0       0.80      0.81      0.80       277
        2.0       0.55      0.54      0.55       123

avg / total       0.72      0.72      0.72       400

[[224  53]
 [ 57  66]]


I thimk the performace of these three algorithms are similar, with precision and recall both around 70%. I prefer using AdaBoost for this dataset because it can predict '2' more accurately than the other two.

Also, i think the data size is too small for trainning the random forest and AdaBoost, since the estimators are large, and i use 48 varibales here instead of 19 variables in the dataset, due to dummy varibales. With that much variables, i need more samples.